# 価格は正しいです

今日、商品の価格を推定するためのより複雑なソリューションを構築しています。

1。Day2.0ノートブック：400,000のトレーニングデータを使用してRAGデータベースを作成する
2。Day2.1ノートブック：2dで視覚化します
3。2.2日目ノートブック：3Dで視覚化します
4。Day2.3ノートブック：GPT-4O-MINIでRAGパイプラインを構築してテストする
5。Day2.4ノートブック：（a）ランダムフォレストプライザー（b）すべての価格からの寄付を可能にするアンサンブルプライザーを作成する

うーん！それは一日で乗り越えるのにたくさんのことです！

＃＃ ご注意ください：

私たちはすでに、独自のファインチューニングされたLLMを使用して非常に強力な製品推定器を持っています。ほとんどの人はそれに非常に満足するでしょう！これらの追加の手順を追加する主な理由は、RAGとエージェントワークフローで専門知識を深めることです。

In [ ]:
# 輸入

import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [ ]:
# あなた自身の責任でこれを上げてください！ 10_000は安全です。

MAXIMUM_DATAPOINTS = 20_000

In [ ]:
DB = "products_vectorstore"
client = chromadb.PersistentClient(path=DB)

In [ ]:
collection = client.get_or_create_collection('products')

In [ ]:
CATEGORIES = ['Appliances', 'Automotive', 'Cell_Phones_and_Accessories', 'Electronics','Musical_Instruments', 'Office_Products', 'Tools_and_Home_Improvement', 'Toys_and_Games']
COLORS = ['red', 'blue', 'brown', 'orange', 'yellow', 'green' , 'purple', 'cyan']

In [ ]:
# プレウォーク
result = collection.get(include=['embeddings', 'documents', 'metadatas'], limit=MAXIMUM_DATAPOINTS)
vectors = np.array(result['embeddings'])
documents = result['documents']
categories = [metadata['category'] for metadata in result['metadatas']]
colors = [COLORS[CATEGORIES.index(c)] for c in categories]

In [ ]:
# 3Dを試してみましょう！

tsne = TSNE(n_components=3, random_state=42, n_jobs=-1)
reduced_vectors = tsne.fit_transform(vectors)

In [ ]:

# 3D散布図を作成します
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=3, color=colors, opacity=0.7),
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=1200,
    height=800,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()